In the function `nr_pusch_channel_estimation`, `max_ch` is used to track the maximum channel coefficient value encountered during the channel estimation process. This value is then used in subsequent calculations, such as normalizing the channel estimates and determining the signal-to-noise ratio (SNR). Here is a summarized explanation of its usage:

1. **Initialization**:
   ```c
   int max_ch = 0;
   uint32_t nvar = 0;
   ```

2. **Channel Estimation Loop**:
   - The function loops through the symbols in the uplink resource grid.
   - For each symbol flagged as a DMRS (Demodulation Reference Signal) symbol, the function calls `nr_pusch_channel_estimation` for each layer:
     ```c
     for (int nl = 0; nl < rel15_ul->nrOfLayers; nl++) {
         uint32_t nvar_tmp = 0;
         nr_pusch_channel_estimation(gNB,
                                     slot,
                                     nl,
                                     get_dmrs_port(nl, rel15_ul->dmrs_ports),
                                     symbol,
                                     ulsch_id,
                                     bwp_start_subcarrier,
                                     rel15_ul,
                                     &max_ch,
                                     &nvar_tmp);
         nvar += nvar_tmp;
     }
     ```

3. **Updating max_ch**:
   - Within `nr_pusch_channel_estimation`, the `max_ch` value is updated with the maximum channel coefficient value for each layer and antenna. This updated value is then used in the outer function.

4. **SNR Measurement**:
   - The function measures the SNR using the estimated channel values:
     ```c
     nr_gnb_measurements(gNB, 
                         &gNB->ulsch[ulsch_id], 
                         pusch_vars, 
                         symbol, 
                         rel15_ul->nrOfLayers);
     ```

5. **Signal and Noise Power Calculation**:
   - The function calculates the signal and noise power for each antenna based on the channel estimates:
     ```c
     for (int aarx = 0; aarx < frame_parms->nb_antennas_rx; aarx++) {
         if (symbol == rel15_ul->start_symbol_index) {
             pusch_vars->ulsch_power[aarx] = 0;
             pusch_vars->ulsch_noise_power[aarx] = 0;
         }
         for (int aatx = 0; aatx < rel15_ul->nrOfLayers; aatx++) {
             pusch_vars->ulsch_power[aarx] += signal_energy_nodc(
                 &pusch_vars->ul_ch_estimates[aatx * gNB->frame_parms.nb_antennas_rx + aarx][symbol * frame_parms->ofdm_symbol_size],
                 rel15_ul->rb_size * 12);
         }
         for (int rb = 0; rb < rel15_ul->rb_size; rb++)
             pusch_vars->ulsch_noise_power[aarx] += 
                 n0_subband_power[aarx][rel15_ul->bwp_start + rel15_ul->rb_start + rb] / rel15_ul->rb_size;
     }
     ```

6. **Noise Variance Calculation**:
   - The noise variance is averaged across all symbols, layers, and antennas:
     ```c
     nvar /= (rel15_ul->nr_of_symbols * rel15_ul->nrOfLayers * frame_parms->nb_antennas_rx);
     ```

7. **Time Domain Channel Estimation**:
   - Optionally, the function performs time-domain averaging of the channel estimates if the corresponding flag is set:
     ```c
     if (gNB->chest_time == 1) {
         nr_chest_time_domain_avg(frame_parms,
                                  pusch_vars->ul_ch_estimates,
                                  rel15_ul->nr_of_symbols,
                                  rel15_ul->start_symbol_index,
                                  rel15_ul->ul_dmrs_symb_pos,
                                  rel15_ul->rb_size);
         pusch_vars->dmrs_symbol = get_next_dmrs_symbol_in_slot(rel15_ul->ul_dmrs_symb_pos, 
                                                                rel15_ul->start_symbol_index, 
                                                                rel15_ul->nr_of_symbols);
     }
     ```

8. **Channel Level Computation**:
   - The maximum channel value (`max_ch`) is used to adjust the shift amount for channel estimate extraction, ensuring proper scaling and preventing overflow:
     ```c
     uint8_t shift_ch_ext = rel15_ul->nrOfLayers > 1 ? log2_approx(max_ch >> 11) : 0;
     ```

In summary, `max_ch` is used throughout the `nr_pusch_channel_estimation` function to:
- Track the maximum channel coefficient value across symbols and layers.
- Normalize the channel estimates.
- Calculate the signal and noise power.
- Adjust the scaling factor during channel estimate extraction to ensure numerical stability and accuracy.